In [1]:
from CompactFIPS202 import SHAKE256
from sage.functions.log import logb
import random

# prime=2**64-59=18446744073709551557
# output-length l=1; c=1
# alpha=3,m=3, security_level=32

#definition of the Grendel_permutation

def legendre_symbol(value,p):
    return value^((p-1)/2)

def get_mds_matrix(p,m):
    #get a primitive element
    Fp=FiniteField(p)
    g=Fp(2)
    while g.multiplicative_order()!=p-1:
        g=g+1
    V=matrix([[g^(i*j) for j in range(0,2*m)] for i in range(0,m)])
    V_ech=V.echelon_form()
    MDS=V_ech[:,m:].transpose()
    return MDS
    
def get_round_constants(p,m,security_level,N):
    bytes_per_int=1+ceil(logb(p,2)/8)
    num_bytes=m*N*bytes_per_int
    seed_string="grendel-%i-%i-%i"%(p,m,security_level)
    byte_string=SHAKE256(bytes(seed_string, "ascii"), num_bytes)
    round_constants=[]
    Fp=FiniteField(p)
    for i in range(m*N):
        acc=0
        for j in range(bytes_per_int):
            acc=256*acc+ZZ(byte_string[i*bytes_per_int+j])
        round_constants.append(Fp(acc %p))
    return round_constants

def Grendel_permutation(p,alpha,m,N,state,linear_layer,round_constants):
    new_state=state
    for i in range(0,N):
        for j in range(0,m):
            new_state[j]=(new_state[j])^alpha*(legendre_symbol(new_state[j],p))
        new_state=linear_layer*vector(new_state)
        for j in range(0,m):
            new_state[j]=new_state[j]+round_constants[i*m+j]
    return new_state


# definition of extract_guesses
def extract_guesses(permutation_guess,number_single_guesses):
    single_guesses=[]
    for i in range(0,number_single_guesses):
        guess_intermediate=(permutation_guess>>i)&0b1
        guess_converted=(guess_intermediate*1)+((guess_intermediate-1)*1)
        single_guesses.append(guess_converted)
    return single_guesses

#Finding a preimage for a sponge hash function instantiated with N-round Grendel over F_p^m
p=18446744073709551557
Fp=GF(p)
security_level=32
alpha=3
alpha_1=inverse_mod(alpha,p-1)
m=3
N=6

M=get_mds_matrix(p,m)

round_constants=get_round_constants(p,m,security_level,N)


print("p,alpha,m,N",p,alpha,m,N)

p,alpha,m,N 18446744073709551557 3 3 6


In [2]:
#build the equation for each guess
L=m*(N-1)
guess_limit=2^L

R.<x>=PolynomialRing(Fp)

load("ntl_power_mod.spyx")
print("load: ntl_power_mod.spyx")

Compiling ./ntl_power_mod.spyx...


load: ntl_power_mod.spyx


In [3]:
num_success=0
for k in range(100):
    a=random.randint(0,p)
    x1=Fp(a)
    b=random.randint(0,p)
    x2=Fp(b)
    state_uniform=[x1,x2,Fp(0)]
    H=Grendel_permutation(p,alpha,m,N,state_uniform,M,round_constants)
    h=H[0]
    solution=[ ]
    
    for tt in range(0,2):
        c=random.randint(0,p)
        w1=Fp(c)       
        for permutation_guess in range(0,guess_limit):
            state=[x,w1,Fp(0)]
            single_guesses=extract_guesses(permutation_guess,L)
            new_state=M*vector(state)
            for j in range(m):
                new_state[j]=new_state[j]+round_constants[j]
            for i in range(1,N):
                for j in range(m):
                    new_state[j]=((new_state[j])^alpha)*single_guesses[(i-1)*m+j]
                new_state=M*vector(new_state)
                for j in range(m):
                    new_state[j]=new_state[j]+round_constants[i*m+j]
            equation=new_state[0]-h
            Q=ntl_power_mod(x,Fp.order(),equation)
            R=equation.gcd(Q-x)
            for r in R.roots(multiplicities=False):
                flag=True
                state=[r,w1,Fp(0)]
                new_state=M*vector(state)
                for j in range(m):
                    new_state[j]=new_state[j]+round_constants[j]
                for i in range(1,N):
                    for j in range(m):
                        le=legendre_symbol(new_state[j],p)
                        if le!= single_guesses[(i-1)*m+j]:
                            flag=False
                            break
                        new_state[j]=((new_state[j])^alpha)*le   
                    if flag==True:
                        new_state=M*vector(new_state)
                        for j in range(m):
                            new_state[j]=new_state[j]+round_constants[i*m+j]
                    else:
                        break
                if flag==True:
                    input_state=[0]*m
                    for j in range(m):
                        input_state[j]=(state[j]^alpha_1)*(legendre_symbol(state[j],p))
                    output_state=Grendel_permutation(p,alpha,m,N,input_state,M,round_constants)
                    solution.append([input_state,output_state,h])
                    break
            if len(solution)>0:
                break
        print(solution)
        if len(solution)>0:
            num_success=num_success+1
            break
print(num_success)

[[[1856239268434541265, 11004345080245978645, 0], (4509062438608773474, 2251689553834771362, 2289193210277769847), 4509062438608773474]]
[[[2045200473841417085, 18301336091470815723, 0], (4243125673455910321, 4474061207990225560, 9532930483466493728), 4243125673455910321]]
[[[17864080122510714696, 10614637846283451099, 0], (9990253607945145766, 4642004787667047776, 9893788465968645394), 9990253607945145766]]
[[[8587708027105799772, 7369887430013401187, 0], (2941894522462574864, 10748102348678385673, 5544442190244027340), 2941894522462574864]]
[[[8981714180730232784, 4720645027436772758, 0], (6711519075128275623, 7725480884057483733, 13767804806618437295), 6711519075128275623]]
[[[419172426054720215, 17891648359186403699, 0], (4226075767448398297, 16967158379350410240, 9191547260029763572), 4226075767448398297]]
[]
[]
[[[8529864746267479152, 15997162369969277768, 0], (15041999624370991775, 1996235069787786096, 7456772023386594073), 15041999624370991775]]
[[[5526017718876195396, 29099882

[]
[[[6558462384521151679, 11125756683897077553, 0], (17115610105706206489, 129118502526229217, 9244758389625303099), 17115610105706206489]]
[[[15845526599859653469, 17950590542978566900, 0], (5137898834255818288, 17962594971745517908, 4172996746903530224), 5137898834255818288]]
[[[17233315433158514216, 11397482666476149260, 0], (1904298399807964936, 14910689627256411666, 1140620531828248463), 1904298399807964936]]
[]
[[[14692346266289984200, 7166633361800990499, 0], (4880044818188169131, 11605854680910074306, 4951507437402266588), 4880044818188169131]]
[[[14000021667536150724, 18076099038550154993, 0], (13913565030426877807, 16068902806735466848, 15649490903626506801), 13913565030426877807]]
[[[7167668414561466259, 7423472896375236828, 0], (8184655161544444933, 18067717208865634746, 11454723694407180787), 8184655161544444933]]
[]
[]
[]
[]
[[[3894807284146844543, 16272804465916042094, 0], (7257529625372902498, 2389141419695398493, 9578023262402688218), 7257529625372902498]]
[]
[[[15851